In [1]:
import scanpy as sc

/deltadisk/miniconda3/envs/gears/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/deltadisk/miniconda3/envs/gears/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/deltadisk/miniconda3/envs/gears/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/deltadisk/miniconda3/envs/gears/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/deltadisk/miniconda3/envs/gears/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_

In [2]:
# 指定包含 matrix.mtx.gz, features.tsv.gz, barcodes.tsv.gz 的目录路径
data_dir = './E20250579-01-01_RNA_matrix/Sh-B'  # 请替换为你的实际路径

adata_Sh_B = sc.read_10x_mtx(
    data_dir,                   # 目录路径
    var_names='gene_symbols',   # 通常用基因符号作为变量名
    cache=True                  # 缓存以加速后续读取
)

In [3]:
# 指定包含 matrix.mtx.gz, features.tsv.gz, barcodes.tsv.gz 的目录路径
data_dir = './E20250579-01-01_RNA_matrix/Sh-E'  # 请替换为你的实际路径

adata_Sh_E = sc.read_10x_mtx(
    data_dir,                   # 目录路径
    var_names='gene_symbols',   # 通常用基因符号作为变量名
    cache=True                  # 缓存以加速后续读取
)

In [4]:
# 指定包含 matrix.mtx.gz, features.tsv.gz, barcodes.tsv.gz 的目录路径
data_dir = './E20250579-01-01_RNA_matrix/Sh-H'  # 请替换为你的实际路径

adata_Sh_H = sc.read_10x_mtx(
    data_dir,                   # 目录路径
    var_names='gene_symbols',   # 通常用基因符号作为变量名
    cache=True                  # 缓存以加速后续读取
)

In [5]:
# 指定包含 matrix.mtx.gz, features.tsv.gz, barcodes.tsv.gz 的目录路径
data_dir = './E20250579-01-01_RNA_matrix/SCR'  # 请替换为你的实际路径

adata_SCR = sc.read_10x_mtx(
    data_dir,                   # 目录路径
    var_names='gene_symbols',   # 通常用基因符号作为变量名
    cache=True                  # 缓存以加速后续读取
)

In [6]:
adata_SCR.obs['cell_type'] = 'HUVEC'
adata_Sh_B.obs['cell_type'] = 'HUVEC'
adata_Sh_E.obs['cell_type'] = 'HUVEC'
adata_Sh_H.obs['cell_type'] = 'HUVEC'

In [7]:
adata_SCR.obs['condition'] = 'ctrl'

In [8]:
adata_Sh_H.obs['condition'] = 'YWHAH+ctrl'

In [9]:
adata_Sh_B.obs['condition'] = 'YWHAB+ctrl'

In [10]:
adata_Sh_E.obs['condition'] = 'YWHAE+ctrl'

In [11]:
adata_combined = sc.concat(
    [adata_SCR, adata_Sh_B, adata_Sh_E, adata_Sh_H],
    join='inner',  # 使用基因的交集，确保所有数据集共有的基因
    label='dataset',  # 在obs中添加标识列
    keys=['SCR', 'Sh_B', 'Sh_E', 'Sh_H'],  # 每个数据集的标识
    index_unique='-'  # 使索引唯一，添加数据集前缀
)

In [12]:
adata_combined

AnnData object with n_obs × n_vars = 46439 × 33694
    obs: 'cell_type', 'condition', 'dataset'

In [13]:
sc.pp.normalize_total(adata_combined)
sc.pp.log1p(adata_combined)
sc.pp.highly_variable_genes(adata_combined,n_top_genes=5000, subset=True)

In [14]:
import sys
sys.path.append('../')

from gears import PertData, GEARS

In [15]:
adata_combined.obs['condition']

AAACCCAAGAATCCCT-1-SCR           ctrl
AAACCCAAGGATACAT-1-SCR           ctrl
AAACCCACAAATCAAG-1-SCR           ctrl
AAACCCACAACCACAT-1-SCR           ctrl
AAACCCAGTCTAGTGT-1-SCR           ctrl
                              ...    
TTTGTTGGTACCTATG-1-Sh_H    YWHAH+ctrl
TTTGTTGGTGAGACCA-1-Sh_H    YWHAH+ctrl
TTTGTTGGTGAGGCAT-1-Sh_H    YWHAH+ctrl
TTTGTTGTCATTACGG-1-Sh_H    YWHAH+ctrl
TTTGTTGTCTCGGCTT-1-Sh_H    YWHAH+ctrl
Name: condition, Length: 46439, dtype: object

In [16]:
adata_combined.var['gene_name'] = adata_combined.var.index

In [17]:
adata_combined.write_h5ad("adata_combined.h5ad")

In [18]:
import pickle

split_dict = {
    'train': ['YWHAB+ctrl', 'ctrl'],
    'val': ['YWHAE+ctrl', 'ctrl'],
    'test': ['YWHAH+ctrl', 'ctrl']
}

with open('custom_split.pkl', 'wb') as f:
    pickle.dump(split_dict, f)
    

In [19]:


pert_data = PertData('/deltadisk/zhangrongyu/GEARS/demo/data') # specific saved folder
pert_data.new_data_process(dataset_name = 'private', adata = adata_combined) # specific dataset name and adata object
pert_data.load(data_path = '/deltadisk/zhangrongyu/GEARS/demo/data/private') # load the processed data, the path is saved folder + dataset_name
pert_data.prepare_split(split='custom', split_dict_path='custom_split.pkl')# get data split with seed
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128) # prepare data loader

Found local copy...
Found local copy...
Creating pyg object for each cell in the data...
Creating dataset file...
100%|██████████| 4/4 [00:52<00:00, 13.22s/it]
Done!
Saving new dataset pyg object at /deltadisk/zhangrongyu/GEARS/demo/data/private/data_pyg/cell_graphs.pkl
Done!
Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
[]
Local copy of pyg dataset is detected. Loading...
Done!
Creating dataloaders....
Done!


In [20]:
train_graphs = pdata.dataloader['train_loader'].dataset
train_labels = [g.pert for g in train_graphs]  # 用 perturbation 作标签

NameError: name 'pdata' is not defined